In [2]:
### GLOWORM SWARM OPTIMIZATION EXAMPLE
### LUKE PETERSON 06/30/2016
### INFO: https://en.wikipedia.org/wiki/Glowworm_swarm_optimization
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
from scipy.spatial import distance as dist
import copy
import simpy

In [6]:
##############################################################################
# GSO PARAMETERS 
##############################################################################
dims = 10
num_worms = 20
nturns = 100
lower_bound = 70
influence_factor = 30
max_jitter = 0.2

In [7]:
##############################################################################
# Fitness Function 
##############################################################################
def fitness_function(xy_tuple):
    x = xy_tuple[0] / 2
    y = xy_tuple[1] / 2
    return (np.sin(x**3 + (y - 5)**3)
            + np.cos((y - 5)**2) * 10
            + np.cos(x * 2) * 12 * (y - 5))

In [ ]:
##############################################################################
# GSO FUNCTIONS 
##############################################################################
class Glowworm:
    def __init__(self, env, name, swarm, X=np.array([1, 1]), waypoint = np.array([250,250]),  speed=1.5433, 
                 transRange=100.0, score = 0.0):
        self.env = env
        self.name = name
        self.swarm = swarm  # List of all particles
        self.transmission_range = transRange
        self.speed = speed # 0.015433 -> 3 knots
        self.waypoint = waypoint

        self.X = X
        self.V = np.random.randn(2) * 0.1
        self.score = score # for 
        self.influence_dict = {name: 0.0}

        self.positions = [self.X.copy()]
        self.process = env.process(self.run())

    def keep_in_bounds(x, dims):
        if x < 0:
            return 0
        elif x > dims:
            return dims
        else:
            return x

    def sensing(self): # 
        """accquire the fitness of worm's current position to calculate 'influence radius'."""
        shifted = fitness_function(self.X) + lower_bound
        self.score = shifted / influence_factor

    def influence_matrix(self):
        """If worm j is within worm j's radius, record distance; else 0."""
        for worm in self.swarm:
            distance = dist.euclidean(self.X, worm.X)
            if distance <= worm.score:
                self.influence_dict[worm.name] = distance
            if worm.name == self.name:
                self.influence_dict[self.name] = 0.0

    def next_turn(pop, score, inf_mat):
        """Update each worm's position based on the matrix of influences."""
        new_pop = copy.deepcopy(pop)
        for i in range(num_worms):
            x_move = 0.0
            y_move = 0.0
            for j in range(num_worms):
                if inf_mat[i][j] != 0 and score[i] < score[j]:
                    percent_move = 1 - (inf_mat[i][j] / score[j])
                    x_move += (pop[j][0] - pop[i][0]) * percent_move / 10
                    y_move += (pop[j][1] - pop[i][1]) * percent_move / 10
            
            # Add random jitter
            jitter_x = max_jitter * np.random.rand() * np.random.randint(-1,2)
            jitter_y = max_jitter * np.random.rand() * np.random.randint(-1,2)
            
            new_pop[i][0] += x_move + jitter_x
            new_pop[i][1] += y_move + jitter_y
            
            # Bound checking
            new_pop[i][0] = keep_in_bounds(new_pop[i][0], dims)
            new_pop[i][1] = keep_in_bounds(new_pop[i][1], dims)

        return new_pop

    ##############################################################################
    # SIMPY INTEGRATION
    ##############################################################################
    def glowworm_optimization(self, env, pop, positions, nturns):
        """
        A generator (process) that runs GSO for 'nturns' iterations in a SimPy environment.
        Each iteration is considered one time-step in the simulation.
        """
        for step in range(nturns):
            # Compute glowworm logic
            score = self.get_score(pop)
            inf_mat = self.influence_matrix(pop, score)
            pop = self.next_turn(pop, score, inf_mat)

            # Store positions for later analysis or plotting
            positions.append(copy.deepcopy(pop))

            # You could place logging, resource usage, or other simulation interactions here
            
            # Advance simulated time by 1 time unit
            yield self.env.timeout(1)

    def run(self):
        i = 0
        while True:
            # Compute glowworm logic
            self.sensing()
            inf_mat = self.influence_matrix(pop, score)
            pop = self.next_turn(pop, score, inf_mat)
            
            self.move()
            if i%5 ==0:
                self.broadcast()  # broadcast nbest to neighbors

            # Store positions for later analysis or plotting    
            self.positions.append(self.X.copy()) # take the records of position
            i = i + 1
            yield self.env.timeout(1)


In [ ]:
def starting_points():
    """Initialize the worm positions randomly."""
    return np.random.rand(num_worms, 2) * dims

In [ ]:
def run_gso_simpy():
    """
    Sets up the SimPy environment, runs the glowworm process,
    and returns the recorded positions.
    """
    env = simpy.Environment()

    # Initial population
    pop = starting_points()

    # This list will store the population positions at each turn
    positions = [copy.deepcopy(pop)]

    # Create and start the GSO process
    env.process(glowworm_optimization(env, pop, positions, nturns))

    # Run the simulation
    env.run()

    return positions

In [5]:
##############################################################################
# MAIN (Demo)
##############################################################################
if __name__ == "__main__":
    all_positions = run_gso_simpy()